# Example of using simsopt to optimize simsgeo objects

This notebook demonstrates a silly little example of optimizing a simsgeo curve using simsopt. We start with a curve that has random Fourier coefficients for $R(\phi)$ and $Z(\phi)$, where $(R, \phi, Z)$ are cylindrical coordinates. We minimize the length of the curve, keeping the 0-frequency Fourier mode of $R$ fixed. The result should be a circle.

As of now, one change is needed in simsgeo to get this to work: in `simsgeo/simsgeo/magneticaxis.py`, `set_dofs()` should call `self.invalidate_cache()`.

In [1]:
from simsgeo import StelleratorSymmetricCylindricalFourierCurve, CurveLength
import numpy as np
import sys
sys.path.append('..')
from simsopt import optimizable, LeastSquaresProblem, least_squares_serial_solve

Initialize the simsgeo curve. Simsopt's `optimizable` function adds methods for holding degrees of freedom fixed.

In [2]:
nquadrature = 100
nfourier = 4
nfp = 5
curve = optimizable(StelleratorSymmetricCylindricalFourierCurve(nquadrature, nfourier, nfp))

Initialize the Fourier coefficients to random numbers. The first coefficient is the average major radius, so set it to a positive number.

In [3]:
x0 = np.random.rand(curve.num_dofs()) - 0.5
x0[0] = 3.0
curve.set_dofs(x0)
print('Initial curve dofs: ', curve.get_dofs())

Initial curve dofs:  [ 3.          0.0105896   0.36623945 -0.32220711  0.02087978  0.10188207
  0.1526843  -0.15434564 -0.21810063]


Tell the curve object that the first Fourier mode is fixed, whereas all the other dofs are not.

In [4]:
curve.all_fixed(False)
curve.fixed[0] = True

Presently in simsgeo, the length objective is a separate object rather than as a function of Curve itself, so we must create this 2nd object:

In [5]:
obj = optimizable(CurveLength(curve))
print('Initial curve length: ', obj.J())

Initial curve length:  38.95109509481029


The following assignment is needed for simsopt to recognize that `obj` depends on `curve`. Hopefully this step can be made unnecessary with a change in simsgeo.

In [6]:
obj.depends_on = ['curve']

Create a term in the least-squares objective function, $1.0 * (length - 0.0)^2$:

In [7]:
term1 = (obj, 0.0, 1.0)

Put this term into a least-squares problem object:

In [8]:
prob = LeastSquaresProblem([term1])

Simsopt recognizes that derivative information is available:

In [9]:
prob.dofs.grad_avail

True

Let's view the initial global state vector. It should contain all the initial Fourier modes except the first:

In [10]:
prob.x

array([ 0.0105896 ,  0.36623945, -0.32220711,  0.02087978,  0.10188207,
        0.1526843 , -0.15434564, -0.21810063])

Solve the minimization problem:

In [11]:
least_squares_serial_solve(prob)

Using analytic derivatives
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         7.5859e+02                                    1.63e+03    
       1              2         3.8933e+02      3.69e+02       5.86e-01       1.43e+03    
       2              4         2.3284e+02      1.56e+02       1.47e-01       4.79e+02    
       3              6         2.0191e+02      3.09e+01       7.33e-02       2.58e+02    
       4              7         1.8532e+02      1.66e+01       7.33e-02       1.40e+02    
       5              8         1.8518e+02      1.44e-01       1.47e-01       2.30e+02    
       6              9         1.7989e+02      5.30e+00       3.66e-02       7.56e+01    
       7             10         1.7806e+02      1.83e+00       3.66e-02       3.89e+01    
       8             12         1.7790e+02      1.56e-01       9.16e-03       2.21e+01    
       9             13         1.7787e+02      3.22e-02       

Examine properties of the optimum found:

In [12]:
print('At the optimum, x: ', prob.x)
print(' Final curve dofs: ', curve.get_dofs())
print(' Final curve length:    ', obj.J())
print(' Expected final length: ', 2 * np.pi * x0[0])
print(' objective function: ', prob.objective())

At the optimum, x:  [ 1.07258513e-05  7.82212229e-10 -2.47621830e-10  4.01831025e-06
  1.40171238e-04  2.35033963e-10 -1.00683606e-10  1.45708612e-05]
 Final curve dofs:  [ 3.00000000e+00  1.07258513e-05  7.82212229e-10 -2.47621830e-10
  4.01831025e-06  1.40171238e-04  2.35033963e-10 -1.00683606e-10
  1.45708612e-05]
 Final curve length:     18.84955622808396
 Expected final length:  18.84955592153876
 objective function:  355.30576999569877


All the Fourier modes after the first (the 0-frequency mode) have been reduced towards 0. The final curve length is approximately that of the circle with the expected radius.